# Reading Structured Grids

UXarray supports reading structured grids and representing them as unstructured grids. This user-guide section will discuss how to load in structured grids using UXarray.

In [ ]:
import uxarray as ux
import xarray as xr
import cartopy.crs as ccrs

import warnings

warnings.filterwarnings("ignore")

## Data

For this notebook, we will be using datasets from the [Xarray tutorial](https://docs.xarray.dev/en/stable/generated/xarray.tutorial.open_dataset.html).

In [ ]:
ds_air_temp = xr.tutorial.open_dataset("air_temperature")
ds_air_temp

In [ ]:
ds_ersstv5 = xr.tutorial.open_dataset("ersstv5")
ds_ersstv5

## Grid

A structured grid can be converted into an unstructured grid by using the ``ux.Grid.from_structured()`` class method. An ``xarray.Dataset`` can be passed in, with CF-compliant longitude and latitude coordinates parsed and converted to an unstructured grid.

In [ ]:
uxgrid = ux.Grid.from_structured(ds_air_temp)
uxgrid

You can also manually pass in longitude and latitude coordinates.

In [ ]:
uxgrid = ux.Grid.from_structured(lon=ds_air_temp.lon, lat=ds_air_temp.lat)
uxgrid

In [ ]:
uxgrid.plot(
    title="Structured Grid loaded using UXarray",
    backend="matplotlib",
    width=1000,
    coastline=True,
)

## Dataset

If you have a dataset that contains data variables, you can convert the entire ``xarray.Dataset`` into a ``uxarray.UxDataset`` using the ``ux.UxDataset.from_structured()`` class method. 

In [ ]:
uxds = ux.UxDataset.from_structured(ds_air_temp)
uxds

In [ ]:
uxds["air"][0].plot(
    title="Structured Grid with Data loaded using UXarray",
    backend="matplotlib",
    width=500,
    coastline=True,
)

## Limitations of Structured Grids

Structured grids are a common choice in climate modeling due to their ease of implementation. However, they come with many limitations, which have been a driving factor in the adoption of unstructured grids.



#### Limited Flexibility in Handling Complex Geometries

Structured grids, with their regular and grid-aligned cells, struggle to capture complex geometries. This limitation forces modelers to use a higher number of grid points to approximate complex features, which can lead to increased computational costs and reduced simulation efficiency. Additionally, the inability to precisely represent intricate boundaries can result in inaccuracies in modeling climate processes that are influenced by these geographical features, potentially affecting the reliability of the simulation outcomes.

#### Difficulty in Local Grid Refinement

Climate phenomena like tropical cyclones, atmospheric fronts, and localized convection require high-resolution grids to be accurately modeled. Structured grids make it challenging to refine the grid locally in regions where such detailed resolution is needed without uniformly increasing the grid resolution across the entire model domain. This uniform refinement results in inefficient use of computational resources, as large areas of the model may have unnecessarily high resolution where it is not required.

#### Pole Point Singularities

Pole point singularities are a significant challenge in climate models that utilize structured latitude-longitude grids. In such grid systems, the lines of longitude converge as they approach the Earth's poles, causing the grid cells to become increasingly small near these regions. This convergence leads to several issues:

- **Numerical Instability:** The drastically reduced cell size near the poles can cause numerical methods to become unstable, requiring smaller time steps to maintain accuracy and stability in simulations.

- **Accuracy Problems:** The distortion of grid cell shapes near the poles can lead to inaccuracies in representing physical processes, such as atmospheric circulation and ocean currents, which are critical for realistic climate simulations.



In [ ]:
uxds = ux.UxDataset.from_structured(ds_ersstv5)

Below is a plot of Sea Surface Temperature on a structured grid.

In [ ]:
uxds["sst"][0].plot(
    projection=ccrs.Orthographic(central_latitude=60),
    periodic_elements="split",
    coastline=True,
    grid=True,
    title="SST Near North Pole",
)

If we expose the grid structure, we can observe the singularity at the poles.

In [ ]:
uxds.uxgrid.plot(
    projection=ccrs.Orthographic(central_latitude=60),
    periodic_elements="split",
    width=600,
    linewidth=1,
    coastline=True,
    title="Grid Near NorthPole",
)